In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os

# Import API key
from dotenv import load_dotenv
load_dotenv()
geoapify_key=os.getenv("GEOAPIFY_KEY")


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saurimo,-9.6608,20.3916,23.91,19,9,1.78,AO,1722879413
1,1,bredasdorp,-34.5322,20.0403,8.98,83,11,2.91,ZA,1722879413
2,2,chibougamau,49.9168,-74.3659,17.79,45,75,6.17,CA,1722879413
3,3,duba,27.3513,35.6901,32.70,52,0,2.63,SA,1722879413
4,4,swakopmund,-22.6833,14.5333,13.88,79,24,2.24,NaN,1722879413


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display


# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
)

# Display the map
city_humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]
# Drop any rows with null values
filtered_city_data = narrowed_city_df.dropna()

# Display sample data
filtered_city_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,tolanaro,-25.0319,46.9987,22.04,83,0,2.06,MG,1722879414
84,84,hammerfest,70.6634,23.6821,22.25,46,0,4.12,NO,1722879418
215,215,nazca,-14.8333,-74.9500,24.50,34,0,4.12,PE,1722879428
279,279,dolores hidalgo cuna de la independencia nacional,21.1561,-100.9308,23.81,37,0,1.96,MX,1722879433
295,295,zhangjiakou,40.8100,114.8794,21.71,55,0,2.02,CN,1722879434
318,318,cerkezkoy,41.2850,28.0003,23.10,68,0,4.12,TR,1722879435
338,338,steinbach,50.1667,8.5667,25.85,47,0,2.57,DE,1722879436
352,352,tagab,34.8550,69.6492,26.01,52,0,0.78,AF,1722879438
375,375,kailua-kona,19.6406,-155.9956,25.33,85,0,2.06,US,1722879442
423,423,livingstone,-17.8419,25.8543,21.03,17,0,2.56,ZM,1722879445


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data[["City", "Country", "Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Hotel Name
24,tolanaro,MG,-25.0319,46.9987,
84,hammerfest,NO,70.6634,23.6821,
215,nazca,PE,-14.8333,-74.9500,
279,dolores hidalgo cuna de la independencia nacional,MX,21.1561,-100.9308,
295,zhangjiakou,CN,40.8100,114.8794,
318,cerkezkoy,TR,41.2850,28.0003,
338,steinbach,DE,50.1667,8.5667,
352,tagab,AF,34.8550,69.6492,
375,kailua-kona,US,19.6406,-155.9956,
423,livingstone,ZM,-17.8419,25.8543,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000 
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tolanaro - nearest hotel: Hôtel Mahavokey
hammerfest - nearest hotel: Thon Hotel Hammerfest
nazca - nearest hotel: CASA HACIENDA
dolores hidalgo cuna de la independencia nacional - nearest hotel: LA CAPILLA
zhangjiakou - nearest hotel: 商务宾馆
cerkezkoy - nearest hotel: Akay Otel
steinbach - nearest hotel: Alt Steinbach
tagab - nearest hotel: No hotel found
kailua-kona - nearest hotel: Kona Seaside Hotel
livingstone - nearest hotel: New Fairmount Hotel and Casino
tromso - nearest hotel: Clarion Collection Hotel Aurora
holualoa - nearest hotel: Kona Hotel
andalgala - nearest hotel: Residencial Ayucaba


,City,Country,Lat,Lng,Hotel Name
24,tolanaro,MG,-25.0319,46.9987,Hôtel Mahavokey
84,hammerfest,NO,70.6634,23.6821,Thon Hotel Hammerfest
215,nazca,PE,-14.8333,-74.9500,CASA HACIENDA
279,dolores hidalgo cuna de la independencia nacional,MX,21.1561,-100.9308,LA CAPILLA
295,zhangjiakou,CN,40.8100,114.8794,商务宾馆
318,cerkezkoy,TR,41.2850,28.0003,Akay Otel
338,steinbach,DE,50.1667,8.5667,Alt Steinbach
352,tagab,AF,34.8550,69.6492,No hotel found
375,kailua-kona,US,19.6406,-155.9956,Kona Seaside Hotel
423,livingstone,ZM,-17.8419,25.8543,New Fairmount Hotel and Casino


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 2,
    color = "City",
    alpha = 1,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)


# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)